In [12]:
import json
import random

In [13]:
with open("/Users/arush/downloads/IR_LAMP/Personalized_Product_Rating_validation_input.json", "r") as f:
    json_data = json.load(f)

In [14]:
selected_objects = random.sample(json_data, 200)
selected_json = json.dumps(selected_objects, indent=2)

In [15]:
with open("sampled_rating.json", "w") as f:
    f.write(selected_json)

In [16]:
!pip3 install torch torchvision torchaudio
!pip3 install transformers
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
from tqdm import tqdm
import torch

Python(27350) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


You should consider upgrading via the '/Users/arush/workspace/646proj/LaMP/data/flant5/bin/python3 -m pip install --upgrade pip' command.


Python(27352) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


You should consider upgrading via the '/Users/arush/workspace/646proj/LaMP/data/flant5/bin/python3 -m pip install --upgrade pip' command.


Python(27353) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/Users/arush/workspace/646proj/LaMP/data/flant5/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [17]:
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

In [18]:
with open("/Users/arush/workspace/646proj/LaMP/data/sampled_rating.json", "r") as f:
    json_data = json.load(f)

In [19]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
llm_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")

Python(27354) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
import textwrap
import json
i=0
golds = []
for user in tqdm(json_data):
    i +=1
    corpus = []
    scores = []
    profile = user['profile']
    query = user['input']
    
    input_prompt = "Your task is to answer the query based on user's relevant history interactions. Here’s the query: {}\nHere are user's past interactions: ".format(query)
    past_interactions = []

    for doc in profile:
        corpus.append(doc['text'])
        scores.append(doc['score'])
    corpus_embeddings = model.encode(corpus, convert_to_tensor=True)
    query_embedding = model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=3, score_function=util.dot_score)
    hits = hits[0]
    predictions = {}
    max_len = 2040
    
    for hit in hits:
        new_instance_user_profile = []
        new_past_interaction = "User gave the following review: '{}' and rated the product '{}/5'".format(corpus[hit['corpus_id']], scores[hit['corpus_id']])
        if len(input_prompt) > max_len:
            input_prompt = textwrap.shorten(input_prompt, width=max_len)
        elif (len(input_prompt) + len(new_past_interaction)) < max_len: 
            new_instance_user_profile.append(new_past_interaction)
            input_prompt += "\n" + new_instance_user_profile[0]
        else: 
            break
    inputs = tokenizer(input_prompt, return_tensors="pt")
    outputs = llm_model.generate(**inputs)
    predictions["output"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    predictions["id"] = user["id"]
    golds.append(predictions)

final_json = {}
final_json["task"] = "LaMP_3"
final_json["golds"] = golds

100%|█████████████████████████████████████████| 200/200 [33:48<00:00, 10.14s/it]


In [24]:
with open("pred_prod_rating_personalization.json", "w") as f:
    json.dump(final_json, f)